In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.score import *
from src.train_nn import *

In [3]:
limit_mem()

In [32]:
datadir = '/data/stephan/WeatherBench/5.625deg/'
vars = ['z', 't']
filters = [64, 64, 64, 64, 2]
kernels = [5, 5, 5, 5, 5]
lead_time = 72
batch_size=128
lr=1e-4

In [33]:
z = xr.open_mfdataset(f'{datadir}geopotential_500/*.nc', combine='by_coords')
t = xr.open_mfdataset(f'{datadir}temperature_850/*.nc', combine='by_coords')
ds = xr.merge([z, t], compat='override')

In [34]:
ds_train = ds.sel(time=slice('1979', '2015'))
ds_valid = ds.sel(time=slice('2016', '2016'))
ds_test = ds.sel(time=slice('2017', '2018'))

In [35]:
dic = {var: None for var in vars}
dg_train = DataGenerator(ds_train, dic, lead_time, batch_size=batch_size)
dg_valid = DataGenerator(ds_valid, dic, lead_time, batch_size=batch_size, mean=dg_train.mean,
                         std=dg_train.std, shuffle=False)
dg_test =  DataGenerator(ds_test, dic, lead_time, batch_size=batch_size, mean=dg_train.mean,
                         std=dg_train.std, shuffle=False)
print(f'Mean = {dg_train.mean}; Std = {dg_train.std}')

Loading data into RAM
Loading data into RAM
Loading data into RAM
Mean = <xarray.DataArray 'z' (level: 2)>
array([54108.312  ,   274.52628], dtype=float32)
Coordinates:
  * level    (level) int64 500 500; Std = <xarray.DataArray 'z' (level: 2)>
array([1114.979   ,    5.683259], dtype=float32)
Coordinates:
  * level    (level) int64 500 500


In [36]:
model = build_cnn(filters, kernels, input_shape=(32, 64, len(vars)), activation='elu')
model.compile(keras.optimizers.Adam(lr), 'mse')

In [37]:
model.load_weights('/home/stephan/data/myWeatherBench/predictions/saved_models/fccnn_3d.h5')

In [42]:
model.evaluate_generator(dg_train, verbose=1)

2534/2534 [==============================] - 63s 25ms/step


44.67911972219726

In [43]:
model.evaluate_generator(dg_valid, verbose=1)

69/69 [==============================] - 2s 25ms/step


48.97571379750885

In [44]:
model.evaluate_generator(dg_test, verbose=1)

137/137 [==============================] - 3s 25ms/step


48.30656022907451

In [52]:
dg_train_no_shuffle = DataGenerator(ds_train, dic, lead_time, batch_size=batch_size, 
                                    shuffle=False, mean=dg_train.mean, std=dg_train.std)

Loading data into RAM


In [45]:
preds_test = create_predictions(model, dg_test)
preds_train = create_predictions(model, dg_train_no_shuffle)
preds_valid = create_predictions(model, dg_valid)

In [46]:
rmse_test = compute_weighted_rmse(preds_test, ds).load()
rmse_train = compute_weighted_rmse(preds_train, ds).load()
rmse_valid = compute_weighted_rmse(preds_valid, ds).load()

In [55]:
rmse_train

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    level    int32 500
Data variables:
    z_rmse   float64 593.2
    t_rmse   float64 2.775

In [47]:
rmse_test

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    level    int32 500
Data variables:
    z_rmse   float64 622.0
    t_rmse   float64 2.863

In [49]:
rmse_valid

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    level    int32 500
Data variables:
    z_rmse   float64 627.5
    t_rmse   float64 2.867

In [42]:
model.evaluate_generator(dg_train, verbose=1)

2534/2534 [==============================] - 63s 25ms/step


44.67911972219726

In [43]:
model.evaluate_generator(dg_valid, verbose=1)

69/69 [==============================] - 2s 25ms/step


48.97571379750885

In [44]:
model.evaluate_generator(dg_test, verbose=1)

137/137 [==============================] - 3s 25ms/step


48.30656022907451